In [1]:
#ライブラリのインポート
import glob
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import df_pre as df_pre
import optuna

#lgb用
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold
import optuna.integration.lightgbm as o_lgb

#警告を非表示
import warnings
warnings.simplefilter('ignore')

In [2]:
#ファイルの読み込み
files = glob.glob("../../data/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
data_list
df = pd.concat(data_list)

In [3]:
#変換処理の適用
df = df_pre.data_pre(df)
df

test


,市区町村コード,都道府県名_num,地区名,最寄駅：名称,最寄駅：距離（分）_min,間取り,面積（㎡）_num,建築年_year,建物の構造_num,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点_num,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,
1088170,1204,1,７条通,旭川,16.0,２ＤＫ,45,33.0,5.0,住宅,NaN,近隣商業地域,80.0,300.0,2011.99,未改装,NaN,6.740363
1048405,1106,1,真駒内柏丘,真駒内,25.0,３ＬＤＫ,70,30.0,5.0,住宅,住宅,第１種住居地域,60.0,200.0,2017.50,改装済,NaN,6.875061
1030233,1104,1,菊水１条,菊水,6.0,４ＬＤＫ,105,14.0,5.0,NaN,住宅,第１種住居地域,60.0,200.0,2020.50,未改装,NaN,7.518514
1000080,1101,1,大通西,西１１丁目,4.0,３ＬＤＫ,100,17.0,5.0,住宅,住宅,商業地域,80.0,600.0,2019.99,未改装,NaN,7.792392
1065071,1108,1,大谷地東,ひばりが丘(北海道),9.0,３ＬＤＫ,75,15.0,5.0,住宅,NaN,近隣商業地域,80.0,300.0,2009.50,未改装,NaN,7.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47014161,47208,47,勢理客,NaN,NaN,３ＬＤＫ,70,20.0,5.0,住宅,NaN,第１種住居地域,60.0,200.0,2010.50,未改装,NaN,7.255273
47027826,47208,47,牧港,NaN,NaN,３ＬＤＫ,65,24.0,5.0,住宅,NaN,準工業地域,60.0,200.0,2013.99,改装済,NaN,7.278754
47003754,47208,47,港川,NaN,NaN,３ＬＤＫ,65,17.0,5.0,NaN,その他,商業地域,80.0,400.0,2018.50,改装済,NaN,7.397940


In [4]:
#テストデータの用意
df_train, df_val =train_test_split(df, test_size=0.2)
col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)


In [15]:
#Optunaの実装（テスト）
def objective(trial):
    params = {
        "objective":"regression_l1",    #MAEを指標とした回帰を指定
        "random_seed":1234,             #乱数シードを固定
        "learning_rate":0.01,           #どれだけ1回の学習を次の学習に反映させるか
        "n_estimators":10000,              #学習回数（1round）
        "verbosity":-1,                 #学習過程の非表示/各回最終の学習結果は表示される

        "num_leaves":trial.suggest_int("num_leaves", 2, 100),                             #先端の葉の最大数
        "max_bin":trial.suggest_int("max_bin", 50, 255),                                 #1特徴量の分割条件を増やす
        "max_depth":trial.suggest_int("max_depth", 3, 10),                               #決定木の深さを指定
        "bagging_fraction":trial.suggest_uniform("bagging_fraction", 0.4, 1.0),          #何％の訓練データをランダムに利用するか
        "bagging_freq":trial.suggest_int("bagging_freq", 1, 20),                         #何回ごとにランダムサンプリングをするかを制御
        "feature_fraction":trial.suggest_uniform("feature_fraction", 0.4, 0.9),          #各木を作成するときに何パーセントの特徴量をランダムで利用するか
        "min_data_in_leaf":trial.suggest_int("min_data_in_leaf", 2, 16),                 #各葉に所属する最小データ数
        "min_sum_hessian_in_leaf":trial.suggest_int("min_sum_hessian_in_leaf", 0.001, 10),   #葉を分割するのに必要なhessianの合計値？？->値が小さいと葉を分割し、オーバーフィッティングへ。デフォルト値が0.001で設定おかしい？
    }
    categorical_features = ["地区名", "最寄駅：名称", "間取り", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_eval = lgb.Dataset(val_x, val_y, reference=lgb_train)

    model_lgb = lgb.train(params, lgb_train,
                         valid_sets=lgb_eval,
                         num_boost_round=1000,        #学習回数(1round)->n_estimatorsと被る？？
                         early_stopping_rounds=100,    #何回連続でscoreが改善されなかった場合に打ち切るかの指定
                         verbose_eval=5000)            #何回の学習毎に結果を表示させるか
    y_pred = model_lgb.predict(val_x, num_iteration=model_lgb.best_iteration)
    score=mae(val_y, y_pred)
    return score




In [16]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, timeout=28800)#n_trials=50がdefault、timeout=60と設定すると60秒間試行する
study.best_params #学習結果の表示

[I 2022-01-14 09:14:54,969] A new study created in memory with name: no-name-63f4332d-434d-4607-adba-4699067d4cde


Training until validation scores don't improve for 100 rounds
[5000]	valid_0's l2: 0.014579
[10000]	valid_0's l2: 0.0135831
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 0.0135831


[I 2022-01-14 09:22:40,787] Trial 0 finished with value: 0.07153661351919775 and parameters: {'num_leaves': 56, 'max_bin': 197, 'max_depth': 7, 'bagging_fraction': 0.7269299097981381, 'bagging_freq': 9, 'feature_fraction': 0.722947056533328, 'min_data_in_leaf': 8, 'min_sum_hessian_in_leaf': 8}. Best is trial 0 with value: 0.07153661351919775.


Training until validation scores don't improve for 100 rounds
[5000]	valid_0's l2: 0.0130088
[10000]	valid_0's l2: 0.0118411
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 0.0118411


[I 2022-01-14 09:32:29,560] Trial 1 finished with value: 0.06805188204214459 and parameters: {'num_leaves': 97, 'max_bin': 128, 'max_depth': 9, 'bagging_fraction': 0.7173369518517427, 'bagging_freq': 12, 'feature_fraction': 0.8627983191463305, 'min_data_in_leaf': 3, 'min_sum_hessian_in_leaf': 0}. Best is trial 1 with value: 0.06805188204214459.


...

Training until validation scores don't improve for 100 rounds
[5000]	valid_0's l2: 0.0138133
[10000]	valid_0's l2: 0.0127628
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 0.0127628


[I 2022-01-14 17:09:00,216] Trial 67 finished with value: 0.06827599719309498 and parameters: {'num_leaves': 59, 'max_bin': 193, 'max_depth': 8, 'bagging_fraction': 0.9693166124323044, 'bagging_freq': 1, 'feature_fraction': 0.7235983269470181, 'min_data_in_leaf': 11, 'min_sum_hessian_in_leaf': 5}. Best is trial 20 with value: 0.06777531432169094.


Training until validation scores don't improve for 100 rounds
[5000]	valid_0's l2: 0.013786
[10000]	valid_0's l2: 0.0127145
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 0.0127145


[I 2022-01-14 17:19:45,393] Trial 68 finished with value: 0.06821455843080115 and parameters: {'num_leaves': 97, 'max_bin': 53, 'max_depth': 8, 'bagging_fraction': 0.8882071898211181, 'bagging_freq': 11, 'feature_fraction': 0.5669824347984046, 'min_data_in_leaf': 13, 'min_sum_hessian_in_leaf': 0}. Best is trial 20 with value: 0.06777531432169094.


{'num_leaves': 71,
 'max_bin': 143,
 'max_depth': 8,
 'bagging_fraction': 0.9198293955571775,
 'bagging_freq': 20,
 'feature_fraction': 0.8279016711963055,
 'min_data_in_leaf': 2,
 'min_sum_hessian_in_leaf': 3}

In [17]:
study.best_params

{'num_leaves': 71,
 'max_bin': 143,
 'max_depth': 8,
 'bagging_fraction': 0.9198293955571775,
 'bagging_freq': 20,
 'feature_fraction': 0.8279016711963055,
 'min_data_in_leaf': 2,
 'min_sum_hessian_in_leaf': 3}

In [5]:
#optunaで最適化したパラメータによるlightgbm_交差検証（#2_10000検証/回_7h）
#バリデーション用パラメータ変数
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

#LGB用のパラメータ設定
params = {
    "objective": "regression_l1",
    "metrics": "mae",
    "random_seed":1234,
    "learning_rate":0.01,
    "n_estimators":10000,
    "num_leaves":71,
    "bagging_fraction": 0.9198293955571775,
    "bagging_freq": 20,
    "feature_fraction": 0.8279016711963055,
    "min_data_in_leaf": 2,
    "min_sum_hessian_in_leaf": 3,
    "verbose": -1
}

#各種リスト
valid_scores = []
evaluation_results = {}
models = []

#データ分析の技術_kaggleのチュートリアルより
kf = KFold(n_splits=FOLD, shuffle=True, random_state = 42)
for (tr_idx, va_idx) in kf.split(train_x, train_y):
    tr_x = train_x.iloc[tr_idx]
    va_x = train_x.iloc[va_idx]
    tr_y = train_y.iloc[tr_idx]
    va_y = train_y.iloc[va_idx]

    #LGB用のモデルの作成
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)
    categorical_features = ["地区名", "最寄駅：名称", "間取り", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    
    #モデルの学習を行う
    model = lgb.train(
        params, 
        lgb_train, 
        valid_sets=lgb_eval,
        valid_names=['train', 'valid'],
        evals_result=evaluation_results,
                  num_boost_round=NUM_ROUND,
                  early_stopping_rounds=100,
        verbose_eval=VERBOSE_EVAL,
                  categorical_feature = categorical_features)
   
    y_valid_pred = model.predict(va_x)
    score = mae(va_y, y_valid_pred)
    print(f'MAE: {score}')
    valid_scores.append(score)
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[8646]	train's l1: 0.0766428
MAE: 0.07664280198171869
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[6053]	train's l1: 0.077014
MAE: 0.07701396087868025
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5286]	train's l1: 0.0764952
MAE: 0.07649518316063879
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4540]	train's l1: 0.0761159
MAE: 0.07611594075255358
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5015]	train's l1: 0.0762412
MAE: 0.07624119866910642
CV score: 0.07650181708853954


In [6]:
#提出用データ作業
df_test = pd.read_csv("../../data/test.csv", index_col=0)
df_test = df_pre.data_pre(df_test)
df_test = df_test.drop(columns="取引価格（総額）_log", axis=1)

#結果格納用のデータフレーム
df_pred_result = pd.DataFrame(index=[], columns=[1, 2, 3, 4, 5])
a=1
for model in models:
    predict = model.predict(df_test)
    df_pred_result[a] = predict
    a = a+1
df_test["取引価格（総額）_log"] = df_pred_result.mean(axis='columns').to_numpy()
df_test[["取引価格（総額）_log"]].to_csv("../../data/submit_test.csv")

test
